<a href="https://colab.research.google.com/github/ParthMahadik/MusicGeneration/blob/main/Bidirectional_LSTM_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from music21 import converter, chord, note
import glob
import os
import numpy as np
import pickle
from keras.utils import np_utils

## Data Preprocessing

In [ ]:
!mkdir /content/data

In [ ]:
!rm /content/*.zip

rm: cannot remove '/content/*.zip': No such file or directory


In [ ]:
!wget -P /content http://www.jsbach.net/midi/bwv1007.zip http://www.jsbach.net/midi/bwv1008.zip http://www.jsbach.net/midi/bwv1009.zip http://www.jsbach.net/midi/bwv1010.zip http://www.jsbach.net/midi/bwv1011.zip http://www.jsbach.net/midi/bwv1012.zip

--2021-05-29 14:10:23--  http://www.jsbach.net/midi/bwv1007.zip
Resolving www.jsbach.net (www.jsbach.net)... 98.129.229.34
Connecting to www.jsbach.net (www.jsbach.net)|98.129.229.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7206 (7.0K) [application/zip]
Saving to: ‘/content/bwv1007.zip’

bwv1007.zip         100%[===================>]   7.04K  --.-KB/s    in 0s      

2021-05-29 14:10:23 (584 MB/s) - ‘/content/bwv1007.zip’ saved [7206/7206]

--2021-05-29 14:10:23--  http://www.jsbach.net/midi/bwv1008.zip
Reusing existing connection to www.jsbach.net:80.
HTTP request sent, awaiting response... 200 OK
Length: 8021 (7.8K) [application/zip]
Saving to: ‘/content/bwv1008.zip’

bwv1008.zip         100%[===================>]   7.83K  --.-KB/s    in 0s      

2021-05-29 14:10:23 (818 MB/s) - ‘/content/bwv1008.zip’ saved [8021/8021]

--2021-05-29 14:10:23--  http://www.jsbach.net/midi/bwv1009.zip
Reusing existing connection to www.jsbach.net:80.
HTTP request sent, 

In [ ]:
!rm /content/data/*

rm: cannot remove '/content/data/*': No such file or directory


In [ ]:
!unzip -q bwv1007.zip -d /content/data
!unzip -q bwv1008.zip -d /content/data
!unzip -q bwv1009.zip -d /content/data
!unzip -q bwv1010.zip -d /content/data
!unzip -q bwv1011.zip -d /content/data
!unzip -q bwv1012.zip -d /content/data

In [ ]:
filename = 'cs1-1pre.mid'
file = "/content/data/{}".format(filename)

orig_score = converter.parse(file).chordify()

In [ ]:
orig_score.show('text')

{0.0} <music21.instrument.Violoncello Violoncello>
{0.0} <music21.tempo.MetronomeMark andantino Quarter=80.0>
{0.0} <music21.key.Key of G major>
{0.0} <music21.meter.TimeSignature 4/4>
{0.0} <music21.chord.Chord G2>
{0.25} <music21.chord.Chord D3>
{0.5} <music21.chord.Chord B3>
{0.75} <music21.chord.Chord A3>
{1.0} <music21.chord.Chord B3>
{1.25} <music21.chord.Chord D3>
{1.5} <music21.chord.Chord B3>
{1.75} <music21.chord.Chord D3>
{2.0} <music21.chord.Chord G2>
{2.25} <music21.chord.Chord D3>
{2.5} <music21.chord.Chord B3>
{2.75} <music21.chord.Chord A3>
{3.0} <music21.chord.Chord B3>
{3.25} <music21.chord.Chord D3>
{3.5} <music21.chord.Chord B3>
{3.75} <music21.chord.Chord D3>
{4.0} <music21.chord.Chord G2>
{4.25} <music21.chord.Chord E3>
{4.5} <music21.chord.Chord C4>
{4.75} <music21.chord.Chord B3>
{5.0} <music21.chord.Chord C4>
{5.25} <music21.chord.Chord E3>
{5.5} <music21.chord.Chord C4>
{5.75} <music21.chord.Chord E3>
{6.0} <music21.chord.Chord G2>
{6.25} <music21.chord.Chord 

In [ ]:
notes = []
durations = []

for element in orig_score.flat:
    
    if isinstance(element, chord.Chord):
        notes.append('.'.join(n.nameWithOctave for n in element.pitches))
        durations.append(element.duration.quarterLength)

    elif isinstance(element, note.Note):
        if element.isRest:
            notes.append(str(element.name))
            durations.append(element.duration.quarterLength)
        else:
            notes.append(str(element.nameWithOctave))
            durations.append(element.duration.quarterLength)


In [ ]:
N_NOTES = 10 # shows the starting 10 notes

# take minimum of these 2 in case we have less notes in the song than N_NOTES
n_notes = min(N_NOTES, len(notes))

print("Showing first {} notes:".format(n_notes))

print('\nDuration', 'Pitch')
for ind, nd in enumerate(zip(notes, durations)):
    print(nd[1], '\t', nd[0])
    if (ind > n_notes):
      break

Showing first 10 notes:

Duration Pitch
0.25 	 G2
0.25 	 D3
0.25 	 B3
0.25 	 A3
0.25 	 B3
0.25 	 D3
0.25 	 B3
0.25 	 D3
0.25 	 G2
0.25 	 D3
0.25 	 B3
0.25 	 A3


In [ ]:
DATA_FOLDER = '/content/data'

In [ ]:
intervals = range(1)
SEQ_LEN = 32

In [ ]:
files = glob.glob(os.path.join(DATA_FOLDER, "*.mid"))
print("Starting parsing of {} files to extract data.".format(len(files)))

notes = []
durations = []

print("Parsing files: ")
for idx, file in enumerate(files):
  print("{0} {1}".format(idx + 1, file))

  score = converter.parse(file).chordify()

  for interval in intervals:
    sc = score.transpose(interval)

    notes.extend(['START'] * SEQ_LEN)
    durations.extend([0] * SEQ_LEN)

    for element in sc.flat:
      if isinstance(element, chord.Chord):
        notes.append('.'.join(npitch.nameWithOctave for npitch in element.pitches))
        durations.append(element.duration.quarterLength)

      elif isinstance(element, note.Note):
          if element.isRest:
              notes.append(str(element.name))
              durations.append(element.duration.quarterLength)
          else:
              notes.append(str(element.nameWithOctave))
              durations.append(element.duration.quarterLength)


Starting parsing of 36 files to extract data.
Parsing files: 
1 /content/data/cs1-1pre.mid
2 /content/data/cs3-5bou.mid
3 /content/data/cs2-6gig.mid
4 /content/data/cs2-3cou.mid
5 /content/data/cs5-1pre.mid
6 /content/data/cs2-1pre.mid
7 /content/data/cs1-5men.mid
8 /content/data/cs1-4sar.mid
9 /content/data/cs3-4sar.mid
10 /content/data/cs6-4sar.mid
11 /content/data/cs1-3cou.mid
12 /content/data/cs3-6gig.mid
13 /content/data/cs4-6gig.mid
14 /content/data/cs1-6gig.mid
15 /content/data/cs5-6gig.mid
16 /content/data/cs4-5bou.mid
17 /content/data/cs5-5gav.mid
18 /content/data/cs6-6gig.mid
19 /content/data/cs2-2all.mid
20 /content/data/cs4-2all.mid
21 /content/data/cs4-1pre.mid
22 /content/data/cs3-2all.mid
23 /content/data/cs4-4sar.mid
24 /content/data/cs3-3cou.mid
25 /content/data/cs5-2all.mid
26 /content/data/cs5-4sar.mid
27 /content/data/cs6-5gav.mid
28 /content/data/cs2-4sar.mid
29 /content/data/cs4-3cou.mid
30 /content/data/cs5-3cou.mid
31 /content/data/cs6-2all.mid
32 /content/data/

In [ ]:
# let's have a preview
# upto first SEQ_LEN entries we have only 'START'
print(notes[(SEQ_LEN - 1):(SEQ_LEN + 10)])
print(durations[(SEQ_LEN - 1):(SEQ_LEN + 10)])

print(len(notes), len(durations))

['START', 'G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2', 'D3']
[0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
28813 28813


In [ ]:
!mkdir /content/processed

In [ ]:
PROCESSED_DATA_FOLDER = "/content/processed"
NOTES_FILEN = "notes.hmm"
DURATIONS_FILEN = "durations.hmm"

In [ ]:
# Save the extracted notes:
with open(os.path.join(PROCESSED_DATA_FOLDER, NOTES_FILEN), 'wb') as file_h:
  pickle.dump(notes, file_h)

In [ ]:
# Save the extracted durations
with open(os.path.join(PROCESSED_DATA_FOLDER, DURATIONS_FILEN), 'wb') as file_h:
  pickle.dump(durations, file_h)

In [ ]:
DISTINCTS_FILEN = "distinct_notes_durations.hmm"

In [ ]:
unique_notes = sorted(set(notes))
n_u_notes = len(unique_notes)

unique_durations = sorted(set(durations))
n_u_durations = len(unique_durations)

# To make pickling all this together easy:
# (Note that while deserializing too, this order should be followed)
distincts = [unique_notes, n_u_notes, unique_durations, n_u_durations]

with open(os.path.join(PROCESSED_DATA_FOLDER, DISTINCTS_FILEN), 'wb') as file_h:
  pickle.dump(distincts, file_h)

In [ ]:
LOOKUP_FILEN = "lookups.hmm"

In [ ]:
note_int = {note: idx for (idx, note) in enumerate(unique_notes)}
int_note = {idx: note for (idx, note) in enumerate(unique_notes)}

duration_int = {dur: idx for (idx, dur) in enumerate(unique_durations)}
int_duration = {idx: dur for (idx, dur) in enumerate(unique_durations)}

In [ ]:
# So that we can save them all together:
# (Note that this order must be followed during deserialization too.)
lookups = [note_int, int_note, duration_int, int_duration]

In [ ]:
with open(os.path.join(PROCESSED_DATA_FOLDER, LOOKUP_FILEN), 'wb') as file_h:
  pickle.dump(lookups, file_h)

In [ ]:
duration_int

{0: 0,
 Fraction(1, 12): 1,
 Fraction(1, 6): 2,
 0.25: 3,
 Fraction(1, 3): 4,
 Fraction(5, 12): 5,
 0.5: 6,
 Fraction(2, 3): 7,
 0.75: 8,
 1.0: 9,
 1.25: 10,
 Fraction(4, 3): 11,
 1.5: 12,
 1.75: 13,
 2.0: 14,
 2.25: 15,
 2.5: 16,
 3.0: 17,
 4.0: 18}

In [ ]:
int_duration

{0: 0,
 1: Fraction(1, 12),
 2: Fraction(1, 6),
 3: 0.25,
 4: Fraction(1, 3),
 5: Fraction(5, 12),
 6: 0.5,
 7: Fraction(2, 3),
 8: 0.75,
 9: 1.0,
 10: 1.25,
 11: Fraction(4, 3),
 12: 1.5,
 13: 1.75,
 14: 2.0,
 15: 2.25,
 16: 2.5,
 17: 3.0,
 18: 4.0}

In [ ]:
notes_net_inp = []
notes_net_out = []

durations_net_inp = []
durations_net_out = []

In [ ]:
for i in range(len(notes) - SEQ_LEN):
  notes_inp_seq = notes[i:(i + SEQ_LEN)]
  notes_out_seq = notes[i + SEQ_LEN]

  # note_int is the dictionary lookup to get index for a note in our dictionary
  notes_net_inp.append([note_int[in_note] for in_note in notes_inp_seq]) # several notes in notes_inp_seq
  notes_net_out.append(note_int[notes_out_seq]) # only one note as output

  # do the same thing for the durations:

  dur_inp_seq = durations[i:(i + SEQ_LEN)]
  dur_out_seq = durations[i + SEQ_LEN]

  # duration_int is the dictionary lookup to get index for a note in our dictionary
  durations_net_inp.append([duration_int[in_dur] for in_dur in dur_inp_seq])
  durations_net_out.append(duration_int[dur_out_seq])

In [ ]:
n_training_instances = len(notes_net_inp)

# reshape the input into a format compatible with LSTM layers
notes_net_inp = np.reshape(notes_net_inp, (n_training_instances, SEQ_LEN))
durations_net_inp = np.reshape(durations_net_inp, (n_training_instances, SEQ_LEN))
net_input = [notes_net_inp, durations_net_inp]

notes_net_out = np_utils.to_categorical(notes_net_out, num_classes=n_u_notes)
durations_net_out = np_utils.to_categorical(durations_net_out, num_classes=n_u_durations)
net_output = [notes_net_out, durations_net_out]

In [ ]:
net_input

[array([[460, 460, 460, ..., 460, 460, 460],
        [460, 460, 460, ..., 460, 460, 400],
        [460, 460, 460, ..., 460, 400, 206],
        ...,
        [305, 126, 305, ..., 206,   0, 187],
        [126, 305, 238, ...,   0, 187,   0],
        [305, 238, 305, ..., 187,   0, 317]]), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 3],
        [0, 0, 0, ..., 0, 3, 3],
        ...,
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6]])]

In [ ]:
net_output

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]

In [ ]:
MODEL_TRAIN_DATA_FILEN = "training_data.hmm"

In [ ]:
net_inp_out = [net_input, net_output]
 
with open(os.path.join(PROCESSED_DATA_FOLDER, MODEL_TRAIN_DATA_FILEN), 'wb') as file_h:
  pickle.dump(net_inp_out, file_h)

**End of preprocessing of raw data.**

# Making the model

Embeded plus LSTM plus Bidirectional

In [ ]:
with open("/content/processed/training_data.hmm", "rb") as file_td:
  net_input, net_output = pickle.load(file_td)

In [ ]:
with open("/content/processed/distinct_notes_durations.hmm", "rb") as file_nd:
  unique_notes, n_notes, unique_durations, n_u_durations = pickle.load(file_nd)

In [ ]:
from keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape, Bidirectional
from keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from keras.layers import Multiply, Lambda, Softmax
import keras.backend as K 
from keras.models import Model
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):
    """ create the structure of the neural network """

    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x2 = Embedding(n_durations, embed_size)(durations_in) 

    x = Concatenate()([x1,x2])

    x = LSTM(rnn_units, return_sequences=True)(x)
    # x = Dropout(0.2)(x)

    c = Bidirectional(LSTM(rnn_units))(x)
    # c = Dropout(0.2)(c)
                                    
    notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
    durations_out = Dense(n_durations, activation = 'softmax', name = 'duration')(c)
   
    model = Model([notes_in, durations_in], [notes_out, durations_out])

    opti = RMSprop(lr = 0.001)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)

    return model

In [ ]:
# run params
section = 'compose'
run_id = '0006'
music_name = 'cello'

run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])


store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', music_name)

mode = 'build' # 'load' # 

# data params
intervals = range(1)
seq_len = 32

# model params
embed_size = 100
rnn_units = 256


In [ ]:
model = create_network(n_notes, n_u_durations, embed_size, rnn_units)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    46100       input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 100)    1900        input_4[0][0]                    
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
weights_folder = os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))

In [ ]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.fit(net_input, net_output
          , epochs=2000000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )

Epoch 1/2000000
720/720 [==============================] - 11s 15ms/step - loss: 3.3848 - pitch_loss: 2.8782 - duration_loss: 0.5066 - val_loss: 3.5814 - val_pitch_loss: 3.1210 - val_duration_loss: 0.4605
Epoch 2/2000000
720/720 [==============================] - 10s 14ms/step - loss: 3.0481 - pitch_loss: 2.6098 - duration_loss: 0.4383 - val_loss: 3.5069 - val_pitch_loss: 3.0550 - val_duration_loss: 0.4518
Epoch 3/2000000
720/720 [==============================] - 11s 15ms/step - loss: 2.6575 - pitch_loss: 2.3011 - duration_loss: 0.3564 - val_loss: 3.5866 - val_pitch_loss: 3.1108 - val_duration_loss: 0.4758
Epoch 4/2000000
720/720 [==============================] - 11s 15ms/step - loss: 2.2303 - pitch_loss: 1.9579 - duration_loss: 0.2724 - val_loss: 3.6838 - val_pitch_loss: 3.1872 - val_duration_loss: 0.4966
Epoch 5/2000000
720/720 [==============================] - 10s 15ms/step - loss: 1.7908 - pitch_loss: 1.5966 - duration_loss: 0.1943 - val_loss: 3.8897 - val_pitch_loss: 3.3046 - v

In [ ]:
model.save('processed/BiLSTM.h5')

# Prediction

In [ ]:
with open("/content/processed/lookups.hmm", "rb") as file_lp:
  note_to_int, int_to_note, duration_to_int, int_to_duration = pickle.load(file_lp)

with open("/content/processed/distinct_notes_durations.hmm", "rb") as file_nd:
  note_names, n_notes, duration_names, n_durations = pickle.load(file_nd)

In [ ]:
weights_folder = os.path.join(run_folder, 'weights')
weights_file = 'weights.h5'

model = create_network(n_notes, n_durations, embed_size, rnn_units)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    46100       input_5[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 100)    1900        input_6[0][0]                    
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# prediction params
notes_temp=0.5
duration_temp = 0.5
max_extra_notes = 50
max_seq_len = 32
seq_len = 32

# notes = ['START', 'D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3']
# durations = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]


# notes = ['START', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3','F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3']
# durations = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]


notes = ['START']
durations = [0]

if seq_len is not None:
    notes = ['START'] * (seq_len - len(notes)) + notes
    durations = [0] * (seq_len - len(durations)) + durations


sequence_length = len(notes)

In [ ]:
def sample_with_temp(preds, temperature):

    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [ ]:
prediction_output = []
notes_input_sequence = []
durations_input_sequence = []

overall_preds = []

for n, d in zip(notes,durations):
    note_int = note_to_int[n]
    duration_int = duration_to_int[d]
    
    notes_input_sequence.append(note_int)
    durations_input_sequence.append(duration_int)
    
    prediction_output.append([n, d])
    
    if n != 'START':
        midi_note = note.Note(n)

        new_note = np.zeros(128)
        new_note[midi_note.pitch.midi] = 1
        overall_preds.append(new_note)


att_matrix = np.zeros(shape = (max_extra_notes+sequence_length, max_extra_notes))

for note_index in range(max_extra_notes):

    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]

    notes_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    # if use_attention:
    #     att_prediction = att_model.predict(prediction_input, verbose=0)[0]
    #     att_matrix[(note_index-len(att_prediction)+sequence_length):(note_index+sequence_length), note_index] = att_prediction
    
    new_note = np.zeros(128)
    
    for idx, n_i in enumerate(notes_prediction[0]):
        try:
            note_name = int_to_note[idx]
            midi_note = note.Note(note_name)
            new_note[midi_note.pitch.midi] = n_i
            
        except:
            pass
        
    overall_preds.append(new_note)
            
    
    i1 = sample_with_temp(notes_prediction[0], notes_temp)
    i2 = sample_with_temp(durations_prediction[0], duration_temp)
    

    note_result = int_to_note[i1]
    duration_result = int_to_duration[i2]
    
    prediction_output.append([note_result, duration_result])

    notes_input_sequence.append(i1)
    durations_input_sequence.append(i2)
    
    if len(notes_input_sequence) > max_seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]
        
#     print(note_result)
#     print(duration_result)
        
    if note_result == 'START':
        break

overall_preds = np.transpose(np.array(overall_preds)) 
print('Generated sequence of {} notes'.format(len(prediction_output)))

Generated sequence of 33 notes
